# Introduction

In this tutorial, you'll learn how to create interactive maps with the **folium** package.  Along the way, you'll apply your new skills to visualize Boston crime data.

In [ ]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth, Circle
from folium.plugins import HeatMap

We define a function `embed_map()` for displaying interactive maps.  It accepts two arguments: the variable containing the map, and the name of the HTML file where the map will be saved.

This function ensures that the maps are visible [in all web browsers](https://github.com/python-visualization/folium/issues/812).

In [ ]:
# Function for displaying the map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Your first interactive map

We'll begin by creating a relatively simple map with `folium.Map()`.

In [ ]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
embed_map(m_1, 'm_1.html')

We use several arguments to customize the appearance of the map:
- The `location` argument sets the initial center of the map. We use the latitude (42.3601&deg; N) and longitude (-71.0589&deg; E) of the city of Boston.  
- The `tiles` argument changes the styling of the map; in this case, we choose the [OpenStreetMap](https://www.openstreetmap.org/#map=10/42.32/-71.0589) style.  If you're curious, you can find the other options listed [here](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles).
- The `zoom_start` argument sets the initial level of zoom of the map, where higher values zoom in closer to the map.

Feel free to explore by zooming in and out, or by dragging the map in different directions.

# The data

Now, we'll load some crime data to add to the map! 

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the data in a pandas DataFrame `crimes`.  The first five rows of the data are shown below.

In [ ]:
# Load the data
crimes = pd.read_csv("../input/geospatial-course-data/crimes-in-boston/crime.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()

# Bubble map

To create a bubble map, ...  For each criminal incident, we add a circle that's centered at the location.

In [ ]:
# Create a base map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add a bubble map to the base map
for i in range(0,len(crimes)):
    Circle(
        location=[crimes.iloc[i]['Lat'], crimes.iloc[i]['Long']],
        radius=3,
        color='darkred',
        fill=True).add_to(m_2)

# Display the map
embed_map(m_2, 'm_2.html')

We loop over rows of the `crimes` DataFrame, and use `folium.Circle()` to add circles to the map.

Note that `folium.Circle()` takes several arguments:
- `location=[crimes.iloc[i]['Lat'], crimes.iloc[i]['Long']]` supplies the location of the crime (in latitude and longitude).
- `radius=3` sets the radius of the circle.  Note that in a traditional bubble map, these are allowed to vary by xyz.
- `color='darkred'` sets the color of each circle.
- `fill=True` ...

# Heatmap

If our goal is to determine areas with higer crime rates, it might be difficult to gauge from a bubble map.  Heatmap might be better.

To create a heatmap, we use [`folium.plugins.Heatmap()`](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap).  This shows the density of crime in different areas of the city.  

As we'd expect for a big city, most of the crime happens near the center.

In [ ]:
# Create a base map
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(m_3)

# Display the map
embed_map(m_3, 'm_3.html')

We set `data=crimes[['Lat', 'Long']]` as the locations (in latitude and longitude) of the criminal incidents.

The `radius` argument controls the smoothness of the heatmap.  Higher values make the heatmap look smoother (i.e., with fewer gaps).

After creating the heatmap, we use the `add_to()` method to add it to the detailed map of the city.

# Choropleth map

You're interested in understanding how crime varies by police district.

Towards this goal, we'll create a GeoDataFrame where each district is assigned a different row, and the `'geometry'` column contains the geographical boundaries.

In [ ]:
districts = gpd.read_file('../input/geospatial-course-data/police_districts/Police_Districts.shp')
districts = districts[["DISTRICT", "geometry"]]
districts.head()

To prepare to plot the data, we have to convert the GeoDataFrame to a [GeoJSON FeatureCollection](https://en.wikipedia.org/wiki/GeoJSON) with the `__geo_interface__` attribute.


In [ ]:
# Create GeoJSON FeatureCollection containing geographical extents of each district
geo_json_data = districts.set_index('DISTRICT').__geo_interface__

`geo_json_data` is a very large Python dictionary, and you can find a small preview below:

```
{
    'type': 'FeatureCollection',
    'features': [
        {
            'id': 'A15',
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'MultiPolygon',
                'coordinates': [(((-71.07415718153364, 42.390507686248306), 
                                  (-71.07415424746895, 42.390503696137124), ...),)]
            },
           'bbox': (-71.08098977707851, 42.36855783087253, 
                    -71.04732198289226, 42.395115613536824)
        },
        {
            'id': 'A7',
            'type': 'Feature',
            'properties': {},
            'geometry': {
                'type': 'MultiPolygon',
                'coordinates': [(((-70.99644430907341, 42.395567982613706), 
                                  (-70.9964432120769, 42.3955663320891), ...),)]
            },
            'bbox': (-71.04571829638523, 42.34379288691729, 
                     -70.95296107011458, 42.396948318600884)
        },
        ...
    ]
}
```

We'll also create a Pandas Series `plot_dict` that shows the number of crimes in each district.

In [ ]:
plot_dict = crimes.DISTRICT.value_counts()
plot_dict

We'll use both `geo_json_data` and `plot_dict` to create a choropleth map, using the `folium.Choropleth()` class.

In [ ]:
# Create a base map
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=geo_json_data, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_4)

# Display the map
embed_map(m_4, 'm_4.html')

We set `geo_data=geo_json_data` as the GeoJSON object containing the geographical boundaries for each district.

Then, `data=plot_dict` is a Pandas series containing the total number of crimes in each district.  This series will be used to color-code each district.

Next, `key_on="feature.id"` refers to the fact that each district has a corresponding entry in `geo_json_data['features']`, and the values corresponding to the`'id'` key correspond to the index in the `plot_dict` Series.

Finally, `fill_color` sets the color scale, and `legend_name` labels the legend in the top right corner.

# Your turn

you've learned about a lot of different map styles. put your new skills to use ...